# Aquisição e Análise de Medições

A classe `MDMS` utiliza as configurações definidas no arquivo `config.toml` para inicializar o motor de aquisição para preparar o ambiente para geração das medições sintéticas via OpenDSS.

O sistema realiza a geração de dados sintéticos para:
- Potência ativa (P)  
- Potência reativa (Q)  
- Magnitudes de tensão (|V|)





In [0]:
# Widgets
dbutils.widgets.text("sub", "TAQ", "Subestação")
dbutils.widgets.text("feeder", "TAQ03", "Alimentador")
dbutils.widgets.text("uni_tr_mt", "34102241", "Rede BT")
dbutils.widgets.text("random_state", "42", "Semente Aleatória")
sub = dbutils.widgets.get("sub")
feeder = dbutils.widgets.get("feeder")
uni_tr_mt = dbutils.widgets.get("uni_tr_mt")
random_state = dbutils.widgets.get("random_state")
if random_state == "":
    random_state = None
else:
    random_state = int(random_state)

In [0]:
import os
import sys
current_path = os.getcwd()
parent_path = os.path.dirname(current_path)
sys.path.append(parent_path)

In [0]:
# MDMS engine
from Utils.MDMS import MDMS, config
mdms = MDMS(sub=sub, feeder=feeder, uni_tr_mt=uni_tr_mt, random_state=random_state)
meas = mdms.acquire(
    customers = {
        "name": ["_b1_0009089969"],
        "kw_by_phase": [1.50],
        "pf": [0.92],
        "shape_type": ["fixed"],
        "working_period": [(15, 21)],
        "theft_on_training": [False],
    }
)

In [0]:
display(mdms.irregs.drop(columns=["theft_kw", "measured_kw"]))

In [0]:
fig = mdms.plot_ntl()

In [0]:
fig = mdms.plot_topology(
    irreg_buses = mdms.irregs["bus_id"].tolist()
)

# Treinamento do LR (LR)

Treinamento de um modelo MLP voltado à previsão das magnitudes de tensão em um sistema elétrico de baixa tensão. 



In [0]:
from sklearn.linear_model import LinearRegression
from Models.LinearRegression import Training
model = LinearRegression()
inputs, outputs = mdms.training.get("inputs"), mdms.training.get("outputs") 
engine = Training(
    model=model, 
    inputs=inputs, 
    outputs=outputs,
    test_size=0.3,
    random_state=random_state
)
model = engine.train()
mse, r2 = engine.evaluate_model()
print(f'MSE: {mse:.6f}, R²: {r2:.6f}')

In [0]:
engine.fig_histogram

In [0]:
engine.fig_boxplot

In [0]:
from Models.LinearRegression import Detector
inputs, outputs = mdms.inference.get("inputs"), mdms.inference.get("outputs") 
detector = Detector(model=model, inputs=inputs, outputs=outputs)
deltaW = detector.execute()

In [0]:
display(deltaW)
    
    

In [0]:
display(mdms.irregs.drop(columns=["measured_kw", "theft_kw"]))

In [0]:
import numpy as np
import pandas as pd
df = pd.merge(left=mdms.irregs[["bus_id", "num_phases", "kw_by_phase", "pf", "kwh_theft"]], right=deltaW, on="bus_id", how="right").dropna()
df["error"] = np.round(abs((df["kwh_theft"] - df["deltaW"])/df["kwh_theft"]),2)
display(df)

In [0]:
from Utils.PlotFunctions import boxplot
fig = boxplot(np.asarray(detector.residues_list)/1)

In [0]:
from Utils.PlotFunctions import plot_inputs, plot_outputs
i = 356
outputs_ = detector.outputs_
inputs_ = detector.inputs_.iloc[outputs_.index]
y_pred = outputs_.iloc[i].outputs
y_true = outputs_.iloc[i].targets
y_optim = outputs_.iloc[i].optimized
x_input = inputs_.iloc[i].inputs
x_optim = inputs_.iloc[i].optimized
columns = inputs.columns
fig1 = plot_inputs(
    columns = columns, 
    x_input = x_input, 
    x_optim = x_optim
)

In [0]:
fig1.savefig("lr-kw.svg")

In [0]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'cm'   # mantém boa compatibilidade com equações

x_series = pd.Series(data=x_optim, index=columns)
y_series = pd.Series(data=x_input, index=columns)

cols_a = [x for x in x_series.index if '1' in x.split('.') and 'kvar' in x.split('.')]
cols_b = [x for x in x_series.index if '2' in x.split('.') and 'kvar' in x.split('.')]
cols_c = [x for x in x_series.index if '3' in x.split('.') and 'kvar' in x.split('.')]

xa = np.arange(0, len(cols_a)) + 1
xb = np.arange(len(cols_a), len(cols_a) + len(cols_b)) + 1
xc = np.arange(len(cols_a) + len(cols_b), len(cols_a) + len(cols_b) + len(cols_c)) + 1

fig, axes = plt.subplots(1, 3, figsize=(8.5, 2.6))

axes[0].step(xa, y_series[cols_a].values, linestyle='-', color='blue', lw=1.5, where="mid", label='Measurements')
axes[0].step(xa, x_series[cols_a].values, linestyle='-.', color='orange', lw=1.5, where="mid")
axes[0].fill_between(x=xa - 0.5, y1=y_series[cols_a], step='post', color='blue', alpha=0.15)
axes[0].fill_between(x=xa - 0.5, y1=x_series[cols_a], step='post', color='orange', alpha=0.15)
axes[0].grid(alpha=0.2)

axes[1].step(xb, y_series[cols_b].values, linestyle='-', color='blue', lw=1.5, where="mid")
axes[1].step(xb, x_series[cols_b].values, linestyle='-.', color='orange', lw=1.5, where="mid", label='Adjusted')
axes[1].fill_between(x=xb - 0.5, y1=y_series[cols_b], step='post', color='blue', alpha=0.15)
axes[1].fill_between(x=xb - 0.5, y1=x_series[cols_b], step='post', color='orange', alpha=0.15)
axes[1].grid(alpha=0.2)

axes[2].step(xc, y_series[cols_c].values, linestyle='-', color='blue', lw=1.5, where="mid")
axes[2].step(xc, x_series[cols_c].values, linestyle='-.', color='orange', lw=1.5, where="mid")
axes[2].fill_between(x=xc - 0.5, y1=y_series[cols_c], step='post', color='blue', alpha=0.15)
axes[2].fill_between(x=xc - 0.5, y1=x_series[cols_c], step='post', color='orange', alpha=0.15)
axes[2].grid(alpha=0.2)

axes[0].set_ylabel('Active powers (kW)')
axes[1].set_ylabel('Active powers (kW)')
axes[2].set_ylabel('Active powers (kW)')
axes[0].set_xlabel('Monitored nodes')
axes[1].set_xlabel('Monitored nodes')
axes[2].set_xlabel('Monitored nodes')

y_max = np.max(
    [x_series[cols_a].values.max(), y_series[cols_a].values.max(),
    x_series[cols_b].values.max(), y_series[cols_b].values.max(),
    x_series[cols_c].values.max(), y_series[cols_c].values.max(),
    ])+1

for ax in axes:
    ax.tick_params(axis='both')
    ax.set_ylim(0, y_max)
    ax.legend(loc='upper left').get_frame().set_edgecolor('none')

axes[0].set_xticks(np.arange(0, max(xa) + 1, 5))
axes[1].set_xticks(np.arange(min(xb), max(xb) + 1, 5))
axes[2].set_xticks(np.arange(min(xc), max(xc) + 1, 5))
plt.tight_layout()

In [0]:
fig.savefig("lr-kvar.svg")

In [0]:
fig2 = plot_outputs(
    columns = outputs.columns, 
    y_pred = y_pred, 
    y_true = y_true, 
    y_optim = y_optim
)

In [0]:
fig2.savefig("lr-v.svg")